In [1]:
import pandas as pd
import requests as rq

In [3]:
complains_url="https://data.cityofnewyork.us/api/views/erm2-nwe9/rows.csv"
pop_byzip_url = "https://data.cityofnewyork.us/resource/rreq-n6zk.csv"
pop_byborough_url = "https://data.cityofnewyork.us/api/views/h2bk-zmw6/rows.csv?accessType=DOWNLOAD"

#### for some reason pandas ignores nrows() when reading from URL so we use bash magic to expolore schema

In [4]:
%%bash -s "$complains_url" "$pop_byzip_url" "$pop_byborough_url"
echo "++++++++++++++++++++++++++++++ COMPLAINS ++++++++++++++++++++++++++++++++"
wget -qO- $1 | head -2

echo "++++++++++++++++++++++++ POPULATION BY ZIP CODES ++++++++++++++++++++++++"
wget -qO- $2 | head -2

echo "+++++++++++++++++++++++++ POPULATION BY BOROUGH +++++++++++++++++++++++++"
wget -qO- $3 | head -2

++++++++++++++++++++++++++++++ COMPLAINS ++++++++++++++++++++++++++++++++
Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,Street Name,Cross Street 1,Cross Street 2,Intersection Street 1,Intersection Street 2,Address Type,City,Landmark,Facility Type,Status,Due Date,Resolution Description,Resolution Action Updated Date,Community Board,BBL,Borough,X Coordinate (State Plane),Y Coordinate (State Plane),Open Data Channel Type,Park Facility Name,Park Borough,Vehicle Type,Taxi Company Borough,Taxi Pick Up Location,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Latitude,Longitude,Location
39903778,08/02/2018 12:00:00 AM,08/07/2018 12:00:00 AM,DOHMH,Department of Health and Mental Hygiene,Rodent,Condition Attracting Rodents,3+ Family Apt. Building,10075,246 EAST   80 STREET,EAST   80 STREET,3 AVENUE,2 AVENUE,,,ADDRESS,NEW YORK,,N/A,Closed,09/01/2018 05:25:39 PM,The Department of Health and

##### Load files locally
this may not be very efficient, but standard Pandas `read_csv` on URL is loading whole dataset into memory

In [ ]:
%%bash -s "$complains_url" "$pop_byzip_url" "$pop_byborough_url"
echo "++++++++++++++++++++++++++++++ COMPLAINS ++++++++++++++++++++++++++++++++"
wget -O data/complains.csv $1

echo "++++++++++++++++++++++++ POPULATION BY ZIP CODES ++++++++++++++++++++++++"
wget -O data/popul_byzip.csv $2

echo "+++++++++++++++++++++++++ POPULATION BY BOROUGH +++++++++++++++++++++++++"
wget -O data/popul_byborough.csv $3

### Sample data loading & exploration

In [5]:
complains_f = "data/complains.csv"
pop_byzip_f = "data/popul_by_zip.csv"
pop_byborough_f = "data/popul_by_borough.csv"
chunksize = 100000
year=2017

In [6]:
chunks = []
for chunk in pd.read_csv(complains_url, skipinitialspace=True, index_col='Unique Key',
                         usecols=['Unique Key','Created Date','Borough','Incident Zip','City','Complaint Type'],
                         dtype={'Unique Key':int,'Incident Zip':int,'Borough':'category','City':'category','Complaint Type':'category'},
                         na_values=[0, "0", "N/A", "N/A", 0, "N/A", "N/A"],
                         low_memory=False, iterator=True, chunksize=chunksize):
    c = chunk[(pd.to_datetime(chunk['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")]
# lambda: c = chunk.pipe(lambda x: x[(pd.to_datetime(x['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")])
    chunks.append(c)
    print("Added %d rows" %(len(c)))
          
df = pd.concat(chunks, axis=0, ignore_index=True)


KeyboardInterrupt: 

In [ ]:
import io
chunks = []
for chunk in pd.read_csv(io.StringIO(r.content.decode('utf-8')), skipinitialspace=True, index_col='Unique Key',
                         usecols=['Unique Key','Created Date','Borough','Incident Zip','City','Complaint Type'],
                         dtype={'Unique Key':int,'Incident Zip':int,'Borough':'category','City':'category','Complaint Type':'category'},
                         na_values=[0, "0", "N/A", "N/A", 0, "N/A", "N/A"],
                         low_memory=False, iterator=True, chunksize=chunksize):
    # chunks.append(chunk[(pd.to_datetime(chunk['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")])
    # c = chunk[(pd.to_datetime(chunk['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")]
    c = chunk.pipe(lambda x: x[(pd.to_datetime(x['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")])
    chunks.append(c)
    print("Added %d rows" %(len(c)))
    

In [ ]:
iters = 0
chunks = []
for lines in r.iter_lines(decode_unicode='utf-8'):
    print(iters)
    if (iters > 0):
        df = pd.read_csv(StringIO(lines), usecols=[0,1,5,8,16,25], index_col=0, header=None,
                         names=['Unique Key','Created Date','Complaint Type','Incident Zip','City','Borough'],
                         dtype={'Incident Zip': int})
    else:
        df = pd.read_csv(StringIO(lines), header=0, usecols=[0,1,5,8,16,25], index_col=0,
                         names=['Unique Key','Created Date','Complaint Type','Incident Zip','City','Borough'],
                         dtype={'Incident Zip': int})
    
    chunks.append(df)
    iters += 1
    if iters == 10:
        break

df = pd.concat(chunks, axis=0, ignore_index=False, sort=False)
#df.to_csv("data/rows_small.csv")

In [ ]:
df.dtypes

In [ ]:
class UrlReader:
    def __init__(self, url, nrows):
        self.url = url
        self.nrows = nrows
        
    def read():
        lines = []
        with closing(requests.get(url, stream=True)) as r:
            f = (line.decode('utf-8') for line in r.iter_lines())
            yield f
    
#def get_url(url):
#    with closing(requests.get(url, stream=True)) as r:
#        f = (line.decode('utf-8') for line in r.iter_lines())
#        reader = csv.reader(f, delimiter=',', quotechar='"')
#        yield reader

In [ ]:
iter_df=pd.read_csv(StringIO(r.text), skipinitialspace=True, index_col='Unique Key', usecols=['Unique Key','Created Date','Borough','Incident Zip','City','Complaint Type'], low_memory=False, iterator=True, chunksize=chunksize)
#df = pd.concat([chunk[(pd.to_datetime(chunk['Created Date']).dt.year == year) & (chunk['Borough'] != "Unspecified")] for chunk in iter_df])

In [ ]:
df.head(n=5)

In [ ]:
df.dtypes

In [ ]:
df.describe(include = 'all')

In [ ]:
df.info(null_counts=True)

In [ ]:
df_borough = df[['Borough']].drop_duplicates()
df_borough

In [ ]:
df.groupby(['Borough'])['Borough'].count().reset_index(name='cnt').sort_values(by='cnt', ascending=False)

In [ ]:
df_compl_type = df[['Complaint Type']].drop_duplicates()
df_compl_type

In [ ]:
# Borough == Unspecified is less then 0.5% of all records so we are going to drop those records from the DataFrame
# This also can be done at a loading time (see above)
df = df[df['Borough'].map(lambda x: str(x) != "Unspecified")]
df.count()

In [ ]:
df.groupby(['Borough'])['Borough'].count().reset_index(name='cnt').sort_values(by='cnt', ascending=False)

In [ ]:
# There are few complains where City or/and Zip is Nan:
city_null = df['City'].isnull()
zip_null = df['Zip'].isnull()

# Select these cases:
df[zip_null | city_null]

In [ ]:
df = df[df['Zip'].notnull() | df['City'].notnull()]
df.count()

### Assessment part

#### 1. Consider only the 10 most common overall complaint types. For each borough, how many of each of those 10 types were there in 2017?

In [ ]:
# First top 10 complains types:
top10_complains = df.groupby(['Complaint Type'])['Complaint Type'].count().reset_index(name='cnt').sort_values(by='cnt', ascending=False).head(n=10)
top10_complains

In [ ]:
all_merged_with_top10 = pd.merge(top10_complains, df, on='Complaint Type')

In [ ]:
# For each borough, how many of each of those 10 types were there in 2017?
top10_borough_complains = all_merged_with_top10.groupby(['Borough','Complaint Type'])['Complaint Type'].count().reset_index(name='cnt').sort_values(by=['Borough','cnt'], ascending=[True, False])
top10_borough_complains

In [ ]:
# top 10 complains by each borough (this contains exaclt 10 rows per borough):
top10_complains_by_borough = df.groupby(['Borough','Complaint Type'])['Complaint Type'].count().groupby(level='Borough').nlargest(10).reset_index(level=0, drop=True)
top10_complains_by_borough

#### 2. Consider only the 10 most common overall complaint types. For the 10 most populous zip codes, how many of each of those 10 types were there in 2017?


In [ ]:
#1. Find top 10 mst populous zip codes
top10_zip_pop = pd.read_csv(pop_byzip_url, skipinitialspace=True, usecols=['jurisdiction_name','count_participants']).sort_values(by=['count_participants'], ascending=[False]).head(10)
top10_zip_pop

In [ ]:
t_df = pd.merge(top10_zip_pop, df, left_on="jurisdiction_name", right_on='Zip')

In [ ]:
top10_complains_by_zip = t_df.groupby(['Zip','Complaint Type'])['Complaint Type'].count().groupby(level='Zip').nlargest().reset_index(level=0, drop=True)
top10_complains_by_zip

#### 3. Considering all complaint types. Which boroughs are the biggest "complainers" relative to the size of the population in 2017 Meaning, calculate a complaint-index that adjusts for population of the borough

In [ ]:
borough_popul = pd.read_csv(pop_byborough_url, skipinitialspace=True).apply(lambda x: x.astype(str).str.upper())
borough_popul

In [ ]:
borough_compl = df.groupby(['Borough'])['Borough'].count().reset_index(name='complains')
borough_compl

In [ ]:
borough_compl_per_popul_unit = pd.merge(borough_compl, borough_popul, on="Borough")
borough_compl_per_popul_unit['compl_per_1m_popul'] = 1000000*borough_compl_per_popul_unit['complains']/borough_compl_per_popul_unit['Population']
borough_compl_per_popul_unit = borough_compl_per_popul_unit.sort_values(by=['compl_per_1m_popul'], ascending=[False])
borough_compl_per_popul_unit

In [7]:

s = "11219-0283"
s[:5]

'11219'

In [9]:
int(s[:5])

11219

In [15]:
s.replace(r'-[\s\S]*$', "")

'11219-0283'

'11219'